In [ ]:
from lightcurve_fitting import lightcurve, models, fitting, bolometric

# Read the Light Curve
Change the path to the file.
It should have at least these columns: MJD, mag, dmag, filter.
If the columns are not fixed width, you may need to add the keyword `format` (see [`astropy.table.Table.read`](http://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers)).
Most reasonable filter names should be recognized.
Also give the extinction ($E(B-V)$) and the distance modulus to calculate the absolute magnitudes.
If necessary, you can give a second extinction correction for the host galaxy with the `host_ebv` keyword, which is applied at the host redshift.

In [ ]:
lc = lightcurve.LC.read('lightcurve_fitting/example/SN2016bkv.txt')
lc.meta['dm'] = 30.79  # distance modulus
lc.meta['ebv'] = 0.016  # Milky Way extinction
lc.meta['host_ebv'] = 0.  # host galaxy extinction
lc.meta['redshift'] = 0.002  # redshift
lc.show_in_notebook()

To make a bolometric light curve and/or color curves, skip to the [last section](#Bolometric-Light-Curve).

# Set Up the Parameters for the Fit
If you only want to fit a subset of the data, you can do that here.
Choose your model and initialize it with your light curve.
(See the package documentation for information on the various choices of model.)
You also need to give some kind of guesses for the parameters.
Define your priors and starting guesses for each parameter.
The walkers start in the boundaries defined by `p_lo` and `p_up` but can walk out to the prior boundaries.

In [ ]:
lc_early = lc.where(MJD_min=57468., MJD_max=57485.)
model = models.ShockCooling2(lc_early)
priors = [
    models.UniformPrior(0., 100.),
    models.UniformPrior(0., 100.),
    models.UniformPrior(0., 100.),
    models.UniformPrior(57468., 57468.7),
]
p_lo = [20., 2., 20., 57468.5]
p_up = [50., 5., 50., 57468.7]

# Run the Fit
You can modify the number of walkers and the number of steps here.
I'm starting them with numbers that are probably **too small** so that you can test that everything works.
You can save the results to a file using `save_sampler_as='filename.npy'`.

When the fit is done, check the plots to make sure they have converged during the burn-in period.

In [ ]:
sampler = fitting.lightcurve_mcmc(lc_early, model, priors=priors, p_lo=p_lo, p_up=p_up,
                                  nwalkers=10, nsteps=100, nsteps_burnin=100, show=True)

# View the Results
This makes a corner plot with the posterior distributions and the $1\sigma$ credible intervals, as well as a plot showing the best-fit models compared to the data in the upper right.
You can save this plot with `save_plot_as='filename.pdf'`.

In [ ]:
fig, ax_corner, ax_model = fitting.lightcurve_corner(lc_early, model, sampler.flatchain)

# Check the Validity Times
The shock cooling models are only valid for temperatures above 0.7 eV = 8120 K (Sapir & Waxman 2017), so you should check that you have not included observations where the model goes below that.
If you have, you should rerun the fit without those points.
If you used the Rabinak & Waxman option, the model fails even earlier, but you will have to check that manually.

In [ ]:
p_mean = sampler.flatchain.mean(axis=0)
t_max = model.t_max(p_mean)
print(t_max)
if lc_early['MJD'].max() > t_max:
    print('Warning: your model is not valid for all your observations')

# Bolometric Light Curve and Color Curves

You can also make a bolometric light curve from the photometry table you defined in the first section.
The light curve is divided into epochs (defined by the `bin` argument to `calculate_bolometric`), and processed four different ways:
- Fitting the Planck function using `scipy.curve_fit`. This is very fast but may not give reliable uncertainties. The columns `temp`, `radius`, `dtemp`, and `dradius` come from this fit.
  - The Stefan-Bolzmann law gives the total bolometric luminosity, `lum` and `dlum`.
  - Integrating the Planck function between $U$ and $I$ band (observed) gives `L_opt`.
- Fitting the Planck function using an MCMC routine. This is slower, depending on how many walkers (`nwalkers`) and steps (`burnin_steps` and `steps`) you use, but gives more robust uncertainties. The columns `temp_mcmc`, `radius_mcmc`, `dtemp0`, `dtemp1`, `dradius0`, and `dradius1` come from this fit. My convention for non-Gaussian uncertainties is that 0 is the lower uncertainty and 1 is the upper uncertainty.
  - Integrating the Planck function between $U$ and $I$ band (observed) gives `L_mcmc`, `dL_mcmc0`, and `dL_mcmc1`.
- Directly integrating the observed SED, assuming 0 flux outside of $U$ to $I$. Use this if you do not want to assume the SED is a blackbody. This yields the column `L_int`.

The MCMC routine saves a corner plot for each fit in the folder you specify (`outpath`).
I highly recommend looking through these to make sure the fits converged.
If they didn't, try adjusting the number of burn-in steps (`burnin_steps`).
To save the table, give `save_table_as='filename.table'` as an argument to `calculate_bolometric`.
To save the plot, give `save_plot_as='filename.pdf'` as an argument to `plot_bolometric_results`.

Beware of the units I'm using:
- Temperatures are in kilokelvins (kK).
- Radii are in thousands of solar radii ($1000R_\odot$).
- Luminosities are in watts (W). $1\,\mathrm{W} = 10^7\,\mathrm{erg}\,\mathrm{s}^{-1}$

Optionally, you can calculate one or more colors at each epoch by giving the argument `colors` to `calculate_bolometric`).
These get saved in the same output table in four columns per color, e.g., for $B-V$:
- the color itself, `B-V`,
- the uncertainty on the color, `d(B-V)`,
- whether the color is a lower limit, `lolims(B-V)` (i.e., $B$ was an upper limit), and
- whether the color is an upper limit, `uplims(B-V)` (i.e., $V$ was an upper limit).

The color curves can be plotted conveniently with `plot_color_curves`.

In [ ]:
t = bolometric.calculate_bolometric(lc, outpath='./SN2016bkv_bolometric', colors=['B-V', 'g-r', 'r-i'])
t.show_in_notebook()

In [ ]:
fig1 = bolometric.plot_bolometric_results(t)
fig2 = bolometric.plot_color_curves(t)